In [1]:
from keras.models import load_model
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle

Using TensorFlow backend.
/home/varun/environments/pyimagesearch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/varun/environments/pyimagesearch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/varun/environments/pyimagesearch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/va

In [2]:
test_df = pd.read_csv("../input/test.csv")
train_df = pd.read_csv("../input/diuresis_train.csv")

In [3]:
test_d = test_df.Diuresis.values
train_d = train_df.iloc[:, 1].values

In [4]:
def find_closest_record(record):
    record = np.broadcast_to(record, train_d.shape)    
    distances = np.abs(np.subtract(record, train_d))
    
    idx = np.argsort(distances)[0]
    
    return train_df.iloc[idx, :].values[0]

In [5]:
mod_test = []
for i in tqdm(range(test_df.shape[0]), total = test_df.shape[0]):
    value = test_df.Diuresis.iloc[i]
    closest_id = find_closest_record(value)
    
    mod_record = train_df[train_df.people_ID == closest_id]
    mod_test.append(mod_record.values)

100%|██████████| 14498/14498 [00:27<00:00, 521.58it/s]


In [6]:
mod_test = np.squeeze(np.array(mod_test))[:, 2:]

In [7]:
f = open("../output/ss.pkl", "rb")
ss = pickle.load(f)
f.close()

In [8]:
x_test = ss.transform(mod_test)

In [9]:
x_test = x_test[..., None]

In [10]:
model = load_model("../models/lstm.h5")

W0323 12:31:56.824867 140141025560384 deprecation_wrapper.py:119] From /home/varun/environments/pyimagesearch/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0323 12:31:56.845446 140141025560384 deprecation_wrapper.py:119] From /home/varun/environments/pyimagesearch/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0323 12:31:56.857125 140141025560384 deprecation_wrapper.py:119] From /home/varun/environments/pyimagesearch/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0323 12:31:57.193157 140141025560384 deprecation_wrapper.py:119] From /home/varun/environments/pyimagesearch/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name t

In [11]:
y_pred = model.predict(x_test, batch_size = 256)

In [16]:
sub = {
    "people_ID" : test_df.people_ID,
    "Diuresis-27" : np.squeeze(y_pred)
}

In [17]:
sub_df = pd.DataFrame(sub, columns = ["people_ID", "Diuresis-27"])

In [18]:
sub_df.to_csv("../output/submission_task_two.csv", index = False)